In [35]:
import pandas as pd
import numpy as np

import re
import string

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words as nltk_words
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize

import spacy
import en_core_web_sm

nltk.download('stopwords', download_dir='../../Resources')
nltk.download('punkt', download_dir='../../Resources')
nltk.download('wordnet', download_dir='../../Resources')
nltk.download('omw-1.4', download_dir='../../Resources')
nltk.download('words', download_dir='../../Resources')
nltk.download('averaged_perceptron_tagger', download_dir='../../Resources')

[nltk_data] Downloading package stopwords to ../../Resources...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to ../../Resources...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to ../../Resources...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to ../../Resources...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to ../../Resources...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     ../../Resources...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk.data.path.append('../../Resources')

In [36]:
nlp = en_core_web_sm.load()

# Hyposesis:
- remove all the punctuation and stopwords 
- remove all the punctuation, stopwords and lemmatize
-------------- 
_potential problem is that number of keywords is pretty high. Can tackle that in 2 ways:
split the data into 2 data points with random sampling, or just by half, or using manual labelling. Or just drop random sub-set of words, or not random_
- named entity extraction
--------------
_potential problem, same entity for many outputs, e.g. ('War in Ukraine: Taking cover in a town under attack',
 'Jeremy Bowen was on the frontline in Irpin, as residents came under Russian fire while trying to flee.')
 (the only entity at the beginning is Ukraine). When keep only >= 3 entities, dataset becomes pretty small_

In [21]:
data = pd.read_csv('../../Data/bbc_news.csv')
data.head(5)

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [95]:
data = data.drop_duplicates(subset=['title']).drop_duplicates(subset=['guid']).drop_duplicates(subset=['description']).reset_index(drop=True)
data.head(5)

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [11]:
STOPWORDS = stopwords.words('english')
pd.Series(STOPWORDS)

0             i
1            me
2            my
3        myself
4            we
         ...   
174     weren't
175         won
176       won't
177      wouldn
178    wouldn't
Length: 179, dtype: object

In [16]:
data.loc[100, 'title']

'Doctors call on Wales not to pick Francis following head injury against England'

In [17]:
lemmatizer = WordNetLemmatizer()
sent = data.loc[100, 'title']
pos_tag(word_tokenize(sent))

[('Doctors', 'NNS'),
 ('call', 'VBP'),
 ('on', 'IN'),
 ('Wales', 'NNP'),
 ('not', 'RB'),
 ('to', 'TO'),
 ('pick', 'VB'),
 ('Francis', 'NNP'),
 ('following', 'VBG'),
 ('head', 'JJ'),
 ('injury', 'NN'),
 ('against', 'IN'),
 ('England', 'NNP')]

In [28]:
def transform(sentence):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word, tag in pos_tag(word_tokenize(sentence)):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = lemmatizer.lemmatize(word, wntag)
        lemmas.append(lemma)
    return ' '.join(lemmas)

In [29]:
def clear(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(f'[{re.escape(string.punctuation)}]', '', sentence)
    return ' '.join(filter(lambda w: w not in STOPWORDS, nltk.tokenize.word_tokenize(sentence)))

In [33]:
transform(clear(data.loc[100, 'title']))

'doctor call wale pick francis follow head injury england'

In [43]:
doc = nlp(data.loc[100, 'title'])
[(X.text, X.label_) for X in doc.ents]

[('Wales', 'GPE'), ('Francis', 'PERSON'), ('England', 'GPE')]

# Punctuation and stopwords

In [105]:
data_ps = data[['title', 'description']].copy()
data_ps.columns = ['prompt', 'completion']
data_ps['prompt'] = data_ps['prompt'].apply(clear).apply(lambda x: x.replace(' ', ',') + '\n\n###\n\n')
data_ps['completion'] = data_ps['completion'] + ' END'
data_ps

,prompt,completion
0,"ukraine,angry,zelensky,vows,punish,russian,atr...",The Ukrainian president says the country will ...
1,"war,ukraine,taking,cover,town,attack\n\n###\n\n","Jeremy Bowen was on the frontline in Irpin, as..."
2,"ukraine,war,catastrophic,global,food\n\n###\n\n",One of the world's biggest fertiliser firms sa...
3,"manchester,arena,bombing,saffie,roussoss,paren...",The parents of the Manchester Arena bombing's ...
4,"ukraine,conflict,oil,price,soars,highest,level...",Consumers are feeling the impact of higher ene...
...,...,...
8348,"rugby,league,world,cup,pundits,heap,praise,eng...",BBC pundits heap praise on England after their...
8349,"rugby,world,cup,england,victorious,fans,given,...",England's closely fought victory against Franc...
8350,"rugby,league,world,cup,england,cruise,victory,...",Watch highlights as England cruise to a 60-6 v...
8351,"james,maddison,leicesters,england,hopeful,fail...",With England boss Gareth Southgate watching Le...


In [106]:
data_ps['prompt'].apply(lambda x: len(x.split(','))).describe()

count    8353.000000
mean        7.551658
std         2.017285
min         1.000000
25%         6.000000
50%         7.000000
75%         9.000000
max        18.000000
Name: prompt, dtype: float64

In [107]:
data_ps.to_json('./output/punctuation_stopwords.jsonl', orient='records', lines=True)
data_ps.to_csv('./output/punctuation_stopwords.csv', index=False)

# Punctuation, stopwords and lemmatization

In [108]:
data_psl = data[['title', 'description']].copy()
data_psl.columns = ['prompt', 'completion']
data_psl['prompt'] = data_psl['prompt'].apply(clear).apply(transform).apply(lambda x: x.replace(' ', ',') + '\n\n###\n\n')
data_psl['completion'] = data_psl['completion'] + ' END'
data_psl

,prompt,completion
0,"ukraine,angry,zelensky,vow,punish,russian,atro...",The Ukrainian president says the country will ...
1,"war,ukraine,take,cover,town,attack\n\n###\n\n","Jeremy Bowen was on the frontline in Irpin, as..."
2,"ukraine,war,catastrophic,global,food\n\n###\n\n",One of the world's biggest fertiliser firms sa...
3,"manchester,arena,bomb,saffie,roussoss,parent,h...",The parents of the Manchester Arena bombing's ...
4,"ukraine,conflict,oil,price,soar,highest,level,...",Consumers are feeling the impact of higher ene...
...,...,...
8348,"rugby,league,world,cup,pundit,heap,praise,engl...",BBC pundits heap praise on England after their...
8349,"rugby,world,cup,england,victorious,fan,give,wa...",England's closely fought victory against Franc...
8350,"rugby,league,world,cup,england,cruise,victory,...",Watch highlights as England cruise to a 60-6 v...
8351,"james,maddison,leicester,england,hopeful,fail,...",With England boss Gareth Southgate watching Le...


In [109]:
data_psl.to_json('./output/punctuation_stopwords_lemmatization.jsonl', orient='records', lines=True)
data_psl.to_csv('./output/punctuation_stopwords_lemmatization.csv', index=False)

# Named Entity Extraction

In [110]:
data_nee = data[['title', 'description']].copy()
data_nee.columns = ['prompt', 'completion']
data_nee['prompt'] = data_nee['prompt'].apply(lambda x: ','.join([e.text for e in nlp(x).ents]) + '\n\n###\n\n')
data_nee['completion'] = data_nee['completion'] + ' END'
data_nee

,prompt,completion
0,"Ukraine,Angry Zelensky,Russian\n\n###\n\n",The Ukrainian president says the country will ...
1,Ukraine\n\n###\n\n,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine\n\n###\n\n,One of the world's biggest fertiliser firms sa...
3,"Manchester Arena,Saffie Roussos's\n\n###\n\n",The parents of the Manchester Arena bombing's ...
4,"Ukraine,2008\n\n###\n\n",Consumers are feeling the impact of higher ene...
...,...,...
8348,"Rugby League World Cup,England,10,Samoa\n\n###...",BBC pundits heap praise on England after their...
8349,"Rugby World Cup,France\n\n###\n\n",England's closely fought victory against Franc...
8350,"Rugby League World Cup,Samoa\n\n###\n\n",Watch highlights as England cruise to a 60-6 v...
8351,"James Maddison,World Cup\n\n###\n\n",With England boss Gareth Southgate watching Le...


In [111]:
data_nee = data_nee[data_nee['prompt'].apply(lambda x: len(x.split(','))) >= 3]
data_nee

,prompt,completion
0,"Ukraine,Angry Zelensky,Russian\n\n###\n\n",The Ukrainian president says the country will ...
6,"Ukraine,UK,50,Ukrainian\n\n###\n\n","The home secretary says she is ""surging capaci..."
8,"Fourth,Scotland,Wales\n\n###\n\n",Five things you need to know about the coronav...
9,"Protests,Russia,thousands\n\n###\n\n","People have been held in 53 cities, from St Pe..."
15,"Ukraine,West,Putin\n\n###\n\n",Several US presidents have failed to get the m...
...,...,...
8316,"Phil Foden,Manchester City,2027\n\n###\n\n",Phil Foden signs a new contract with Mancheste...
8319,"Brighton,Thomas Frank,Ivan Toney,England World...",Brentford boss Thomas Frank says Ivan Toney sh...
8324,"Belgorod,11,Russian\n\n###\n\n",Two attackers opened fire on recruits who had ...
8346,"Reece James,Chelsea,World Cup\n\n###\n\n",England and Chelsea defender Reece James looks...


In [112]:
data_nee.to_json('./output/named_entity_extraction_3.jsonl', orient='records', lines=True)
data_nee.to_csv('./output/named_entity_extraction_3.csv', index=False)